In [1]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from sklearn.tree import DecisionTreeRegressor
from tqdm.notebook import tqdm
import optuna
optuna.logging.set_verbosity(optuna.logging.CRITICAL)

In [2]:
%%time
# Loading Stock Prices
path = "../input/jpx-tokyo-stock-exchange-prediction/"
df_prices = pd.read_csv(f"{path}train_files/stock_prices.csv")
df_prices = df_prices[~df_prices["Target"].isnull()]
prices = pd.read_csv(f"{path}supplemental_files/stock_prices.csv")
df_prices = pd.concat([df_prices, prices])
df_prices = df_prices[df_prices.Date>="2021-10-01"]
df_prices.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 2244531 to 111999
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   RowId             200000 non-null  object 
 1   Date              200000 non-null  object 
 2   SecuritiesCode    200000 non-null  int64  
 3   Open              199431 non-null  float64
 4   High              199431 non-null  float64
 5   Low               199431 non-null  float64
 6   Close             199431 non-null  float64
 7   Volume            200000 non-null  int64  
 8   AdjustmentFactor  200000 non-null  float64
 9   ExpectedDividend  601 non-null     float64
 10  SupervisionFlag   200000 non-null  bool   
 11  Target            200000 non-null  float64
dtypes: bool(1), float64(7), int64(2), object(2)
memory usage: 18.5+ MB
CPU times: user 4.22 s, sys: 809 ms, total: 5.03 s
Wall time: 6.79 s


In [3]:
def fill_nans(prices):
    prices.set_index(["SecuritiesCode", "Date"], inplace=True)
    prices.ExpectedDividend.fillna(0,inplace=True)
    prices.ffill(inplace=True)
    prices.fillna(0,inplace=True)
    prices.reset_index(inplace=True)
    return prices

In [4]:
%%time
df_prices = fill_nans(df_prices)
prices = fill_nans(prices)
pd.options.display.float_format = '{:,.6g}'.format
df_prices.describe()

CPU times: user 177 ms, sys: 2.28 ms, total: 180 ms
Wall time: 183 ms


,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,Target
count,"200,000","200,000","200,000","200,000","200,000","200,000","200,000","200,000","200,000"
mean,"5,883.05","2,767.44","2,801.61","2,727.87","2,762.4","616,690",0.999947,0.066339,-0.000681474
std,"2,389.81","4,339.88","4,394.16","4,275.42","4,330.9",2.54446e+06,0.00558683,2.07026,0.0234367
min,"1,301",34,35,33,34,0,0.25,0,-0.355
25%,"3,915.75","1,047","1,060","1,032","1,045","31,400",1,0,-0.011976
50%,"6,200.5","1,867","1,888","1,840","1,862","97,300",1,0,-0.000689893
75%,"7,938.25","3,070","3,110","3,025","3,065","369,000",1,0,0.0103668
max,"9,997","82,000","82,060","79,100","80,030",3.13148e+08,1,297,0.597907


In [5]:
# Utilities 

def calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    weights_mean = weights.mean()
    df = df.sort_values(by='Rank')
    purchase = (df['Target'][:portfolio_size]  * weights).sum() / weights_mean
    short    = (df['Target'][-portfolio_size:] * weights[::-1]).sum() / weights_mean
    return purchase - short

def calc_spread_return_sharpe(df, portfolio_size=200, toprank_weight_ratio=2):
    grp = df.groupby('Date')
    min_size = grp["Target"].count().min()
    if min_size<2*portfolio_size:
        portfolio_size=min_size//2
        if portfolio_size<1:
            return 0, None
    buf = grp.apply(calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio, buf

def add_rank(df, col_name="pred"):
    df["Rank"] = df.groupby("Date")[col_name].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

In [6]:
## By Yuike - https://www.kaggle.com/code/ikeppyo/examples-of-higher-scores-than-perfect-predictions

# This function adjusts the predictions so that the daily spread return approaches a certain value.
        
def adjuster(df):
    def calc_pred(df, x, y, z):
        return df['Target'].where(df['Target'].abs() < x, df['Target'] * y + np.sign(df['Target']) * z)

    def objective(trial, df):
        x = trial.suggest_uniform('x', 0, 0.2)
        y = trial.suggest_uniform('y', 0, 0.05)
        z = trial.suggest_uniform('z', 0, 1e-3)
        df["Rank"] = calc_pred(df, x, y, z).rank(ascending=False, method="first") - 1 
        return calc_spread_return_per_day(df, 200, 2)

    def predictor_per_day(df):
        study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=SD))#5187
        study.optimize(lambda trial: abs(objective(trial, df) - 3), 3)
        return calc_pred(df, *study.best_params.values())

    return df.groupby("Date").apply(predictor_per_day).reset_index(level=0, drop=True)

def _predictor_base(feature_df):
    return model.predict(feature_df[feats])

def _predictor_with_adjuster(feature_df):
    df_pred = feature_df.copy()
    df_pred["Target"] = model.predict(feature_df[feats])
    return adjuster(df_pred).values.T

In [7]:
np.random.seed(0)
feats = ["Close"]
max_score = 0
max_depth = 0
for md in tqdm(range(3,40)):
    model = DecisionTreeRegressor( max_depth=md ) # Controlling the overfit with max_depth parameter
    model.fit(df_prices[feats],df_prices["Target"])
    predictor = _predictor_base
    prices["pred"] = predictor(prices)
    score, buf = calc_spread_return_sharpe(add_rank(prices))
    if score>max_score:
        max_score = score
        max_depth = md
        
model = DecisionTreeRegressor( max_depth=max_depth )
model.fit(df_prices[feats],df_prices["Target"])
print(f'Max_deph={max_depth} : Sharpe Ratio Score base -> {max_score}')

  0%|          | 0/37 [00:00<?, ?it/s]

Max_deph=30 : Sharpe Ratio Score base -> 2.9393630429647564


In [8]:
# Controlling the Sharpe Ratio Score (≃3)
predictor = _predictor_with_adjuster
err = 1
maxSD = 3683
for SD in tqdm(range(maxSD,4000)):
    prices["pred"] = predictor(prices)
    score, buf = calc_spread_return_sharpe(add_rank(prices))
    if abs(score-3)<=err and score<3:
        err=abs(score-3)
        maxSD = SD
        print(f'{maxSD} Sharpe Ratio Score with adjuster -> {score}')
        
SD = maxSD

  0%|          | 0/317 [00:00<?, ?it/s]

3683 Sharpe Ratio Score with adjuster -> 2.999997159148049


In [9]:
%%time
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for prices, options, financials, trades, secondary_prices, sample_prediction in iter_test:
    prices = fill_nans(prices)
    prices.loc[:,"pred"] = predictor(prices)
    prices = add_rank(prices)
    rank = prices.set_index('SecuritiesCode')['Rank'].to_dict()
    sample_prediction['Rank'] = sample_prediction['SecuritiesCode'].map(rank)
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
CPU times: user 194 ms, sys: 4.01 ms, total: 198 ms
Wall time: 256 ms
